In [1]:
%pip install transformers
%pip install hazm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#load model

import tensorflow as tf
import numpy as np
import itertools
from transformers import TFAutoModelForMaskedLM, AutoTokenizer

model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=False, output_attentions=False)

/opt/homebrew/Caskroom/miniforge/base/envs/datastuff/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [3]:
def calculate_sentence_score(sentence_tokens,k=1000,epsilon=0.000001):
    sentence_score = 1000000000000
    sentence = ' '.join(sentence_tokens)
    # inputs = tokenizer(sentence, return_tensors="tf")
    # input_ids = inputs["input_ids"]
    for i,token in enumerate(sentence_tokens):
    # for i in range(len(input_ids)):
        found = False
        # token =  tokenizer.convert_ids_to_tokens([input_ids[i]])
        # token = sentence_tokens[i]
        # masked_input_ids = tf.tensor_scatter_nd_update(input_ids, [[0, i]], [tokenizer.mask_token_id])
        masked_text = ' '.join(sentence_tokens[:i])+' [MASK] '+' '.join(sentence_tokens[i+1:])

        
        input_ids = tokenizer.encode(masked_text, return_tensors="tf")
        # inputs = tokenizer(masked_text, return_tensors="tf")
        # print(token)
        # print(outputs)
        logits = logits = model.__call__(input_ids)[0]
        mask_token_index = tf.where(input_ids == tokenizer.mask_token_id)[0, 1]
        mask_token_logits = logits[0, mask_token_index, :]
        top_k_values, top_k_indices = tf.math.top_k(mask_token_logits, k=k)
        predicted_token_indices = top_k_indices.numpy()
        probabilities = tf.nn.softmax(top_k_values).numpy()
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_indices)
        for j,predicted_token in enumerate(predicted_tokens):
            if predicted_token == token:
                sentence_score *= probabilities[j]
                found = True
                break
        if not found:
            sentence_score *= epsilon
        # print(token)
        # for i in range(k):
        #     print(predicted_tokens[i],probabilities[i])
        # print('----')
    return sentence_score

In [4]:
def clean_permutations(permutations):
    return permutations

In [6]:
import hazm
posTagger = hazm.POSTagger(model = 'resources/pos_tagger.model')

In [10]:
import time
def find_ordered_sentence(sentence):
    tokens = tokenizer.tokenize(sentence)
    print(posTagger.tag(tokens))
    permutations = list(itertools.permutations(tokens))
    cleaned_permutations = clean_permutations(permutations)
    scores = []
    for i,permutation in enumerate(cleaned_permutations):
        permutation_sentence = ' '.join(permutation)
        score = calculate_sentence_score(list(permutation))
        print(permutation_sentence)
        print(score)
        scores.append((permutation_sentence,score))
        print(i+1,'/',len(permutations))
    scores.sort(key=lambda x: x[1], reverse=True)
    for score in scores:
        print(score)
    return scores[0][1]

In [11]:
#main code
find_ordered_sentence('من و دوستم رفتیم مدرسه خوردیم کباب بعد برگشتیم با ماشین')

[('من', 'PRON'), ('و', 'CCONJ'), ('دوستم', 'NOUN'), ('رفتیم', 'VERB'), ('مدرسه', 'NOUN'), ('خوردیم', 'VERB'), ('کباب', 'NOUN'), ('بعد', 'CCONJ'), ('برگشتیم', 'VERB'), ('با', 'ADP'), ('ماشین', 'NOUN')]
من و دوستم رفتیم مدرسه خوردیم کباب بعد برگشتیم با ماشین
3.938015201158702e-10
1 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب بعد برگشتیم ماشین با
2.655913246982646e-16
2 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب بعد با برگشتیم ماشین
2.8018387037349984e-18
3 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب بعد با ماشین برگشتیم
1.30390996417635e-08
4 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب بعد ماشین برگشتیم با
4.676232899267952e-19
5 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب بعد ماشین با برگشتیم
2.1950108427480823e-17
6 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب برگشتیم بعد با ماشین
1.381324805189466e-12
7 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب برگشتیم بعد ماشین با
2.9385973160217223e-19
8 / 39916800
من و دوستم رفتیم مدرسه خوردیم کباب برگشتیم با بعد ماشین
5.31009247443

KeyboardInterrupt: 

In [ ]:
#delete this cell

def calculate_sentence_score(sentence_tokens):
    sentence_score = 1
    sentence = ' '.join(sentence_tokens)
    for i,token in enumerate(sentence_tokens):
        found = False
        masked_text = ' '.join(tokens[:i])+' [MASK] '+' '.join(tokens[i+1:])

        inputs = tokenizer(masked_text, return_tensors="tf")
        outputs = model(inputs)
        logits = outputs.logits

        mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
        # mask_token_index = i
        mask_token_logits = logits[0, mask_token_index, :]

        k = 1000

        top_k_values, top_k_indices = tf.math.top_k(mask_token_logits, k=k)
        predicted_token_indices = top_k_indices.numpy()
        probabilities = tf.nn.softmax(top_k_values).numpy()
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_indices)

        for j,predicted_token in enumerate(predicted_tokens):
            if predicted_token == token:
                sentence_score *= probabilities[j]
                # print('found',sentence_score)
                found = True
                break

        epsilon = 0.00001

        if not found:
            sentence_score *= epsilon
            # print('not found',sentence_score)

        # print(token)
        # for i in range(k):
        #     print(predicted_tokens[i],probabilities[i])
        # print('----')

    soft_parameter = 1000000000000

    return sentence_score*soft_parameter


import itertools
text = "وقتی من و دوستم برمیگشتیم از مدرسه "
tokens = tokenizer.tokenize(text)
permutations = list(itertools.permutations(tokens))
scores = []
for i,permutation in enumerate(permutations):
    print(' '.join(permutation))
    scores.append((' '.join(permutation),calculate_sentence_score(list(permutation))))
    print(i+1,'/',len(permutations))
scores.sort(key=lambda x: x[1], reverse=True)
for score in scores:
    print(score)

وقتی من و دوستم برمیگشتیم از مدرسه
1 / 5040
وقتی من و دوستم برمیگشتیم مدرسه از
2 / 5040
وقتی من و دوستم از برمیگشتیم مدرسه
3 / 5040
وقتی من و دوستم از مدرسه برمیگشتیم
4 / 5040
وقتی من و دوستم مدرسه برمیگشتیم از
5 / 5040
وقتی من و دوستم مدرسه از برمیگشتیم
6 / 5040
وقتی من و برمیگشتیم دوستم از مدرسه
7 / 5040
وقتی من و برمیگشتیم دوستم مدرسه از
8 / 5040
وقتی من و برمیگشتیم از دوستم مدرسه
9 / 5040
وقتی من و برمیگشتیم از مدرسه دوستم
10 / 5040
وقتی من و برمیگشتیم مدرسه دوستم از
11 / 5040
وقتی من و برمیگشتیم مدرسه از دوستم
12 / 5040
وقتی من و از دوستم برمیگشتیم مدرسه
13 / 5040
وقتی من و از دوستم مدرسه برمیگشتیم
14 / 5040
وقتی من و از برمیگشتیم دوستم مدرسه
15 / 5040
وقتی من و از برمیگشتیم مدرسه دوستم
16 / 5040
وقتی من و از مدرسه دوستم برمیگشتیم
17 / 5040
وقتی من و از مدرسه برمیگشتیم دوستم
18 / 5040
وقتی من و مدرسه دوستم برمیگشتیم از
19 / 5040
وقتی من و مدرسه دوستم از برمیگشتیم
20 / 5040
وقتی من و مدرسه برمیگشتیم دوستم از
21 / 5040
وقتی من و مدرسه برمیگشتیم از دوستم
22 / 5040
وقتی من و مدرسه از 

KeyboardInterrupt: 